In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from bs4 import BeautifulSoup, element
import time as tm
import os
import requests

In [2]:
chars_to_remove = ",^\()|\"\n"

In [9]:
# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def get_html(collection_id, folder, data):

    # creates a local directory named 'folder' if it doesn't exist
    if not os.path.isdir(folder):
        os.mkdir(folder)

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this gets a unique list of dates and urls so we can avoid duplication
    date_list = pd.to_datetime(data['date'].unique())
    urls = data['url'].unique()

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for url in urls:

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:

            # this creates a local path for the html, it converts URLS+dates
            # into unique simple strings
            path = folder + '/' + dt.strptime(a.text, '%b %d, %Y').strftime('%Y%m%d') + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', url)) + '.html'

            if url == 'http://www.newarkblack.com/queens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017':
                print(path)
                print('https:' + a.attrs['href'])
                print(dt.strptime(a.text, '%b %d, %Y') in date_list)
                print(not os.path.isfile(path))

            # this is where it saves the html ('onclick' in a.attrs is probably redundant)
            if dt.strptime(a.text, '%b %d, %Y') in date_list and not os.path.isfile(path):
                wp = requests.get('https:' + a.attrs['href'])
                f = open(path, 'wb')
                f.write(wp.content)
                f.close()

In [10]:
chosen = pd.read_csv('final_chosen.csv')

In [11]:
get_html(12706, 'chosenSamp', chosen)

chosenSamp/20191008_httpwww.newarkblack.comqueens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017.html
https://wayback.archive-it.org/12706/20191008192629/http://www.newarkblack.com/queens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017/
True
False
chosenSamp/20191012_httpwww.newarkblack.comqueens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017.html
https://wayback.archive-it.org/12706/20191012181003/http://www.newarkblack.com/queens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017/
True
False
chosenSamp/20191020_httpwww.newarkblack.comqueens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-22017.html
https://wayback.archive-it.org/12706/20191020191017/http://www.newarkblack.com/queens-new-york-annual-pop-shop-artist-market-set-held-multiple-dates-nov-25-2017-december-23-2

KeyboardInterrupt: 

In [13]:
ctr = 0
folder = 'chosenSamp'
chosen['cleaned'] = None

for index, row in chosen.iterrows():

    path = folder + '/' + ''.join(re.findall('\d+', row.date)) + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', row.url)) + '.html'

    f = open(path, 'rb')
    soup = BeautifulSoup(f.read(), "lxml")
    articles = soup.find_all("article")

    if len(articles) == 0:
        articles = soup.find_all(id = 'op-content')

    if len(articles) == 1: 
        ctr = ctr + 1
        text = articles[0].text.translate ({ord(c): "" for c in chars_to_remove})
        chosen.loc[index, 'text'] = text
        chosen.loc[index, 'cleaned'] = 1
    else:
        chosen.loc[index, 'cleaned'] = 0

    f.close()

    tm.sleep(.1)


In [14]:
print(ctr)

1832


In [15]:
chosen

,Unnamed: 0,date,domain,url,text,cleaned
0,16,2019-10-08,www.newarkblack.com,http://www.newarkblack.com,NewarkBlack.com - Newark African American News...,0
1,17,2019-10-08,www.newarkblack.com,http://www.newarkblack.com,�,0
2,18,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/?s={search_term_str...,You searched for {search_term_string} - Newark...,0
3,19,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/advertise,Advertise - NewarkBlack.com CLOSE Home Adverti...,0
4,20,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/category/news,News Archives - NewarkBlack.com CLOSE Home Adv...,0
...,...,...,...,...,...,...
2408,4553,2019-11-02,www.mypaperonline.com,https://www.mypaperonline.com/first-library-wi...,Share on FacebookFollow on FacebookAdd to Goog...,1
2409,4554,2019-11-02,www.mypaperonline.com,https://www.mypaperonline.com/hackettstowns-he...,Share on FacebookFollow on FacebookAdd to Goog...,1
2410,4557,2019-11-02,www.mypaperonline.com,https://www.mypaperonline.com/dont-miss-these-...,Share on FacebookFollow on FacebookAdd to Goog...,1
2411,4565,2019-11-02,www.mypaperonline.com,http://www.mypaperonline.com/10th-anniversary-...,Share on FacebookFollow on FacebookAdd to Goog...,1


In [18]:
chosen.drop('Unnamed: 0', axis=1).to_csv('cleaned1.csv')